In [ ]:
%matplotlib widget

import os
import pathlib
import pprint
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

from Visualizer.DataSet import DataSet
from Visualizer.Monitoring.Persistence.ProfilerSession import ProfilerSession
from Visualizer.Plotting.Plot import Plot

In [ ]:
def get_energy_dict(id_list):
    energy_dict = {}
    source_directory = pathlib.Path().parent.absolute()
    project_directory = f"{source_directory}/.."
    root_directory = f"{project_directory}/.."
    resources_directory = f"{root_directory}/Resources"
    database = f"{resources_directory}/Database.sqlite"

    # Load all profiler sessions
    complete_data_set = DataSet(ProfilerSession.load_all(database))
    for profiler_session_id in id_list:

        data_set = DataSet([profiler_session for profiler_session in complete_data_set.data if profiler_session.id == profiler_session_id])
#         energy_consumption = next(reversed(data_set.data[0].energy_consumption()['Node'].values()))
        energy_consumption = data_set.mean_energy_consumption()

        label = data_set.data[0].summary['Label']
        if 'kernel' in data_set.data[0].profile:
            n = data_set.data[0].profile['kernel']
            if n == 1:
                label += " Unoptimized"
            else:
                label += " Optimized"
        if not label in energy_dict:
            energy_dict[label] = []
        if energy_consumption > 0:
            energy_dict[label].append(energy_consumption)
        print(label)
    return energy_dict


In [ ]:
# id_list = list(range(195, 255)) + list(range(296, 536))
# id_list = list(range(195, 255)) + list(range(296, 446)) + list(range(476, 536))
# unified_memory = list(range(446, 476))
# lavamd_particle = list(range(225, 255)) + list(range(326, 356))
# lava_particle_nw_unified = list(range(225, 255)) + list(range(296, 356)) + list(range(446, 476))
# bfs_kmeans = list(range(195, 225)) + list(range(356, 386))
jacobi = list(range(476, 536))
energy_dict = get_energy_dict(jacobi)

In [ ]:
def autolabel(ax, rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

In [ ]:
def make_bar(energy_dict, l=True):
    no_action = []
    minmax = []
    system = []
    maxfreq = []
    minmaxrank = []
    minmaxscaling = []
    no_action_std = []
    minmax_std = []
    system_std = []
    maxfreq_std = []
    minmaxrank_std = []
    minmaxscaling_std = []
    
    applicaties = []
    avg = 0
    for energy in energy_dict:
        label = energy.replace('Matrix Multiply', 'MatrixMultiply').split(' ')
        
        avg = np.mean(energy_dict[energy])
        std = np.std(energy_dict[energy])

        if len(label) == 1 or (label[1] == 'Optimized' or label[1] == 'Unoptimized'):
            if len(label) != 1 and (label[1] == 'Optimized' or label[1] == 'Unoptimized'):
                applicaties.append(label[0] + '\n' + label[1])
            else:
                applicaties.append(label[0])
            no_action.append(avg)
            no_action_std.append(std)
            continue

        if(label[2] == 'MinMax)'):
            minmax.append(avg)
            minmax_std.append(std)
        elif(label[2] == 'System)'):
            system.append(avg)
            system_std.append(std)
        elif(label[2] == 'Ranked'):
            minmaxrank.append(avg)
            minmaxrank_std.append(std)
        elif(label[2] == 'Scaled'):
            minmaxscaling.append(avg)
            minmaxscaling_std.append(std)
        elif(label[2] == 'Max'):
            maxfreq.append(avg)
            maxfreq_std.append(std)

    ind = np.arange(len(no_action))
    width = 0.1

    fig, ax = plt.subplots()
    rects1 = ax.bar(ind - width * 3, np.round(no_action, 1), width, yerr=no_action_std,
                    label='No action')
    rects2 = ax.bar(ind - width * 2, np.round(minmax, 1), width, yerr=minmax_std,
                    label='MinMax')
    rects3 = ax.bar(ind - width, np.round(system, 1), width, yerr=system_std,
                    label='System')
    rects4 = ax.bar(ind, np.round(maxfreq, 1), width, yerr=maxfreq_std,
                    label='Max Frequency')
    rects5 = ax.bar(ind + width, np.round(minmaxrank, 1), width, yerr=minmaxrank_std,
                    label='Ranked MinMax')
    rects6 = ax.bar(ind + width * 2, np.round(minmaxscaling, 1), width, yerr=minmaxscaling_std,
                    label='Scaled MinMax')


    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Energy(J)')
    ax.set_title('Energy Consumption')
    ax.set_xticks(ind)
    names = [x.replace('Profiler', '') for x in applicaties]
    ax.set_xticklabels(names)
    ax.legend()
    
    if (l):
        autolabel(ax, rects1, "left")
        autolabel(ax, rects2, "center")
        autolabel(ax, rects3, "right")
        autolabel(ax, rects4, "left")
        autolabel(ax, rects5, "center")
        autolabel(ax, rects6, "right")

    fig.tight_layout()

    plt.show()

In [ ]:
make_bar(energy_dict, False)